# MapEXIF-Colab
In Google Colab, read the EXIF information (GPS data) of images stored in a specific folder and visualize it on a map.

## Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install folium
!pip install exifread==2.3.2
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=81cef161611107315f96dab7304e8ae0604de8a097fa042133010fa9c054825c
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize-matplotlib


## A function to extract location information from the EXIF data of images taken with an iPhone

In [ ]:
import exifread

def read_exif_from_image(image_path):
    """Read EXIF data from an image file"""
    try:
        with open(image_path, "rb") as f:
            exif_data = exifread.process_file(f)
        return exif_data
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return None
    except IOError:
        print(f"Failed to read file: {image_path}")
        return None


def dms_to_decimal(dms):
    """Convert a Degree-Minute-Second (DMS) list to decimal coordinates"""
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)


def get_coordinates(exif_data):
    """Extract latitude and longitude from EXIF tags and return them as a tuple"""
    try:
        pre_latitude = eval(exif_data["GPS GPSLatitude"].printable)
        pre_longitude = eval(exif_data["GPS GPSLongitude"].printable)
        latitude = dms_to_decimal(pre_latitude)
        longitude = dms_to_decimal(pre_longitude)
        return (latitude, longitude)
    except KeyError:
        print("Required GPS information is missing.")
        return None

In [ ]:
import glob
import os
import numpy as np

# Extract GPS coordinates from HEIC images in the directory
img_directory = '/content/drive/MyDrive/experiment/picture2025/'
gps_pos = []
name_list = []
location_list = []
altitude_list = []
for image_path in sorted(
    glob.glob(os.path.join(img_directory, '*.HEIC'))):

    exif_data = read_exif_from_image(image_path)
    if exif_data:
        coordinates = get_coordinates(exif_data)
        altitude = get_altitude(exif_data)
        if coordinates:
            basename_without_ext = os.path.splitext(os.path.basename(image_path))[0]    # File name
            location_list.append(coordinates)
            altitude_list.append(altitude)
            name_list.append(basename_without_ext)
        else:
            print("Failed to retrieve GPS information.")
    else:
        print("Failed to read EXIF data.")

# Output file name and GPS coordinates
i = 0
dist = 0
cum_dist_list = []
total_dist = 0
latlon1 = []
latlon2 = []
for name, location, alt in zip(name_list, location_list, altitude_list):
    latlon1 = latlon2
    latlon2 = location
    print("FILE:%s, LAT:%.5f LON:%.5f" %
          (name, location[0], location[1]))